<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация-и-очистка" data-toc-modified-id="Лемматизация-и-очистка-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация и очистка</a></span></li><li><span><a href="#Разделение-на-выборки-и-борьба-с-дисбалансом" data-toc-modified-id="Разделение-на-выборки-и-борьба-с-дисбалансом-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Разделение на выборки и борьба с дисбалансом</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#test" data-toc-modified-id="test-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>test</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация комментариев на позитивные и негативные

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Входные данные:**

- Набор данных с разметкой о токсичности правок.:

https://code.s3.yandex.net/datasets/toxic_comments.csv

Признак - `text` содержит текст комментария

Целевой признак - `toxic`

**Цель:**

Обучить модель классифицировать текстовые комментарии на позитивные и негативные. 

**Задача:**

- Построить модель со значением метрики качества *F1* не меньше 0.75.

**Условия:**

- Для выполнения проекта применять *BERT* необязательно

**Этапы**

1. Загрузка и подготовка данные.
2. Обучение разных моделей. 
3. Выводы.

## Подготовка

Загружаем библиотеку для обработки текста

In [1]:
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

/bin/bash: {sys.executable}: command not found
/bin/bash: {sys.executable}: command not found


In [2]:
import pandas as pd 
import numpy as np 
import spacy
import sys

from sklearn.pipeline import Pipeline
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

nltk.download('wordnet')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df.duplicated().sum()

0

Пропусков и дубликатов нет

In [7]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Токсичны только 11.3% комментариев. Мы обнаружили серьёзный дисбаланс при исследовании данных.

### Лемматизация и очистка

Применим SpaCy

In [9]:
def lemmatize_text(text):
    text = text.lower()
    re_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    lemm_text = nlp(re_text)
    
    return " ".join([token.lemma_ for token in lemm_text])

In [10]:
df['lemm_text'] = df['text'].apply(lemmatize_text)

In [11]:
df['lemm_text'].head()

0    explanation why the edit make under my usernam...
1    d aww   he match this background colour I m se...
2    hey man   I m really not try to edit war   it ...
3       more I can t make any real suggestion on im...
4    you   sir   be my hero   any chance you rememb...
Name: lemm_text, dtype: object

### Разделение на выборки и борьба с дисбалансом

In [13]:
X = df.drop('toxic',axis=1)
y = df['toxic']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True,stratify=y)

In [15]:
X_train.shape[0]/X.shape[0]

0.75

In [16]:
X_test.shape[0]/X.shape[0]

0.25

Проверим распределение классов в выборках

In [17]:
y_train.value_counts(normalize=True)

0    0.898384
1    0.101616
Name: toxic, dtype: float64

In [18]:
y_test.value_counts(normalize=True)

0    0.8984
1    0.1016
Name: toxic, dtype: float64

**Дисбаланс**

In [19]:
def down_up_sample(features, target, fraction, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_down_up = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones]*repeat)
    target_down_up = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones]*repeat)
    
    features_down_up_sampled, target_down_up_sampled = shuffle(
        features_down_up, target_down_up, random_state=12345)
    
    return features_down_up_sampled, target_down_up_sampled

features_upsampled, target_upsampled  = down_up_sample(X_train, y_train, 0.6, 5)

Применим TfidfVectorizer и обучим её на train

In [20]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords,analyzer='word') 
tf_idf = count_tf_idf.fit_transform(features_upsampled['lemm_text']) 
tf_idf.shape

(125097, 96310)

In [21]:
tf_test = count_tf_idf.transform(X_test['lemm_text'])

Убрали частые неинформативные слова.

## Обучение

### LogisticRegression

Подбор параметров

In [23]:
clf_lr = LogisticRegression(random_state=12345, class_weight='balanced')
params_lr = {'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10]}
grid_lr = GridSearchCV(clf_lr, params_lr, cv=3, scoring='f1')
grid_lr.fit(tf_idf, target_upsampled)
predict_lr= grid_lr.predict(tf_idf)
f1_lr = f1_score(target_upsampled, predict_lr).round(3)

grid_lr.best_params_, f1_lr

({'C': 10, 'solver': 'lbfgs'}, 0.988)

### DecisionTreeClassifier

In [28]:
best_model_dtc = None
best_result_dtc = 0
best_depth_dtc = 0
for depth in range(1, 15):
    model_dtc = RandomForestClassifier(random_state=12345, max_depth=depth, class_weight='balanced_subsample') 
    model_dtc.fit(tf_idf,target_upsampled) 
    predict_dtc = model_dtc.predict(tf_idf) 
    result_dtc = f1_score(target_upsampled, predict_dtc)
    if result_dtc > best_result_dtc:
        best_model_dtc = model_dtc
        best_result_dtc = result_dtc
        best_depth_dtc = depth
f1_dtc = best_result_dtc.round(3)

best_depth_dtc, f1_dtc

(14, 0.813)

### RandomForestClassifier

In [29]:
best_model_rfc = None
best_result_rfc = 0
best_depth_rfс=0
for depth in range(1, 15):
    model_rfс = RandomForestClassifier(random_state=12345, max_depth=depth, class_weight='balanced_subsample') 
    model_rfс.fit(tf_idf,target_upsampled) 
    predict_rfс = model_rfс.predict(tf_idf) 
    result_rfс = f1_score(target_upsampled, predict_rfс)
    if result_rfс > best_result_rfc:
        best_model_rfс = model_rfс
        best_result_rfc = result_rfс
        best_depth_rfс = depth
f1_rfc = best_result_rfc.round(3)

best_depth_rfс, f1_rfc

(14, 0.813)

**Сравнение моделей**

In [30]:
best_model = pd.DataFrame(data=[('LogisticRegression', f1_lr),
                                ('DecisionTreeClassifier', f1_dtc),
                                ('RandomForestClassifier', f1_rfc)],
                          columns=['model', 'f1_train'])
best_model

,model,f1_train
0,LogisticRegression,0.988
1,DecisionTreeClassifier,0.813
2,RandomForestClassifier,0.813


### test

**LogisticRegression**

In [27]:
model_lr = LogisticRegression(C=10, class_weight='balanced', solver='lbfgs', random_state=12345)
model_lr.fit(tf_idf, target_upsampled)
predict_lr = model_lr.predict(tf_test)
f1_score(y_test, predict_lr).round(3)

0.753

## Выводы

- Загрузили и изучили данные:
    - Пропусков и дубликатов нет.
    - Выявлен дисбаланс классов.
    
- Подготовили данные:
    - Написали функцию для лемматизации и очистки данных от лишних символов.
    - Разделили данные на обучающую и тестовую выборки.
    - Применили методы upsample и downsamle для борьбы с дисбалансом классов.
    - Создали матрицу cо значениями TF-IDF с учётом стоп-слов.
    
- Обучили модели LogisticRegression, CatBoostClassifier и RandomForestClassifier и сравнили результат f1.
    - Рекомендуемая модель LogisticRegression, с наилучшим результатом f1 = 0.988. На test модель показала f1 = 0.753, что удовлетворяет условию.